In [1]:
from helpers.apps.json_logic import apply_json_logic
from helpers.serializers import get_obj_serialized
from helpers.strings import get_obj_from_path, to_flatten_str
from helpers.apps.daily_reports import generate_exported_file
import datetime
from tqdm.notebook import tqdm
import re
import pandas as pd

In [2]:
def ordenar_chave(elemento):
    partes = re.split(r'(\d+)', elemento)
    return (partes[0], int(partes[1]))

In [10]:
def get_direction(record):
    if record.direction:
        possible_path_direct = "reporting__fields__direction__selectoptions__options"
        direction = get_obj_from_path(record.company.custom_options, possible_path_direct)
        return next(item["name"] for item in direction if item["value"] == record.direction)
    else:
        return None

In [4]:
def get_exporter_extra_columns(company):
    return get_obj_from_path(
        company.custom_options, "reporting__exporter__extra_columns"
    )


In [5]:
def get_exporter_extra_columns_parsed_infos(exporter_extra_columns):
    extra_columns = {}
    for extra_column in exporter_extra_columns:
        column_key = extra_column.get("key", "")
        extra_columns[column_key] = extra_column.get("header", "")
    return extra_columns

In [12]:
def get_values_from_reporting_extra_columns(record, extra_columns):
    new_val = {}
    if extra_columns:
        reporting_formatted = get_obj_serialized(record, is_reporting=True)
        for item in extra_columns:
            json_logic = None
            key = item.get("key", "")
            logic = item.get("logic", False)
            header = item.get("header", "")
            is_date = item.get("isDate", False)
            new_val[("Unidade")] = record.company.name
            new_val[("Encontrado Em")] = record.found_at.strftime('%d-%m-%Y %H:%M:%S')
            new_val[("Rodovia")] = record.road_name
            new_val[("Sentido")] = get_direction(record) if record.direction else ''
            new_val[("Km")] = record.km
            new_val[("Km Final")] = record.end_km
            new_val[("Latitude")] = record.point[1]
            new_val[("Longitude")] = record.point[0]
            new_val[("Classe")] = record.occurrence_type.name
            if key and logic:
                try:
                    json_logic = apply_json_logic(logic, reporting_formatted)
                except Exception:
                    pass
            if not json_logic:
                try:
                    json_logic = reporting_formatted.form_data[key]
                except Exception:
                    pass
            if is_date and json_logic:
                json_logic = format_date(json_logic)
            new_val[(header)] = json_logic
    return new_val

In [13]:
# company=Company.objects.get(name__icontains='DER/SP')
companies  = sorted(list(Company.objects.filter(name__icontains='DER/SP').order_by('name').values_list('name',flat=True)),key=ordenar_chave)
companies

['DER/SP - D1',
 'DER/SP - D2',
 'DER/SP - D3',
 'DER/SP - D4',
 'DER/SP - D5',
 'DER/SP - D6',
 'DER/SP - D7',
 'DER/SP - D8',
 'DER/SP - D9',
 'DER/SP - D10',
 'DER/SP - D11',
 'DER/SP - D12',
 'DER/SP - D13',
 'DER/SP - D14']

In [14]:
dados={}
for name in tqdm(companies):
    reps=Reporting.objects.filter(company__name=name,
                                  occurrence_type__name='Correção de marco quilométrico').order_by('number')
#     print(reps.values_list('number',flat=True))
    for rep in reps:
        if rep.occurrence_type.form_fields.get('measurementColumns'):
            report=get_values_from_reporting_extra_columns(rep,rep.occurrence_type.form_fields['measurementColumns'])
            if rep.number not in dados.keys():
                dados[rep.number] = {}
            dados[rep.number] = report
        if rep.occurrence_type.form_fields.get('measurement_columns'):
            report=get_values_from_reporting_extra_columns(rep,rep.occurrence_type.form_fields['measurement_columns'])
            if rep.number not in dados.keys():
                dados[rep.number] = {}
            dados[rep.number] = report

  0%|          | 0/14 [00:00<?, ?it/s]

In [15]:
dados[:10]

TypeError: unhashable type: 'slice'

In [9]:
df = pd.DataFrame.from_dict(dados)

In [10]:
df.transpose()

,Unidade,Encontrado Em,Rodovia,Km,Km Final,Latitude,Longitude,Classe,Quilometro correto,Observações
D1-24000024,DER/SP - D1,08-04-2024 13:34:00,SP-280,23.0,23.0,-23.138908,-48.437154,Correção de marco quilométrico,24,None
D1-24000049,DER/SP - D1,02-05-2024 11:55:37,SP 360,16.225849,16.225849,-23.035812,-46.842944,Correção de marco quilométrico,82,None
D1-24000050,DER/SP - D1,02-05-2024 12:03:15,SP 360,17.370744,17.370744,-23.027512,-46.840298,Correção de marco quilométrico,83,None
D1-24000051,DER/SP - D1,02-05-2024 12:09:49,SP 360,18.511854,18.511854,-23.019335,-46.842335,Correção de marco quilométrico,84,None
D1-24000052,DER/SP - D1,02-05-2024 12:22:53,SP 360,86.511569,86.511569,-23.000022,-46.848234,Correção de marco quilométrico,87,None
...,...,...,...,...,...,...,...,...,...,...
D14-24000159,DER/SP - D14,09-05-2024 18:58:35,SP 385,23.068997,23.068997,-20.195011,-47.952928,Correção de marco quilométrico,23,None
D14-24000160,DER/SP - D14,09-05-2024 19:00:21,SP 385,22.001533,22.001533,-20.200297,-47.944797,Correção de marco quilométrico,22,Ausente de placa do km
D14-24000161,DER/SP - D14,09-05-2024 19:02:37,SP 385,20.973186,20.973186,-20.205375,-47.93695,Correção de marco quilométrico,21,None
D14-24000162,DER/SP - D14,09-05-2024 19:04:40,SP 385,19.901732,19.901732,-20.210563,-47.928665,Correção de marco quilométrico,20,Ausente de placa do km


In [11]:
month=datetime.datetime.now().month
day=datetime.datetime.now().day

In [12]:
df.transpose().to_excel(f'Rel_DERs_{day}_{month}.xlsx')